In [2]:
import os
import sys
from pyspark.sql import SparkSession

# ==============================================================================
# CORREÇÃO OBRIGATÓRIA PARA JAVA 17
# ==============================================================================
# O Java 17+ bloqueia o acesso via reflexão (reflection) que o Spark utiliza.
# Estas flags liberam o acesso aos módulos internos do Java.
# Sem isso, o gateway fecha imediatamente.
# ==============================================================================
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--driver-java-options "'
    '--add-opens=java.base/java.lang=ALL-UNNAMED '
    '--add-opens=java.base/java.io=ALL-UNNAMED '
    '--add-opens=java.base/java.util=ALL-UNNAMED '
    '--add-opens=java.base/java.util.concurrent=ALL-UNNAMED '
    '--add-opens=java.base/sun.nio.ch=ALL-UNNAMED '
    '--add-opens=java.base/sun.security.action=ALL-UNNAMED '
    '-Dspark.driver.memory=512m" ' # Força memória aqui também
    'pyspark-shell'
)

def create_spark_session(app_name="Prototype Transformation NYC Taxi Data"):
    print("⏳ Iniciando sessão Spark (Isso pode demorar se for o primeiro download dos JARs)...")
    
    return SparkSession.builder \
            .appName(app_name) \
            .config("spark.driver.memory", "512m") \
            .config("spark.executor.memory", "512m") \
            .config("spark.jars.ivy", "/tmp/.ivy") \
            .config("spark.hadoop.fs.s3a.endpoint", os.getenv("MINIO_ENDPOINT")) \
            .config("spark.hadoop.fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID")) \
            .config("spark.hadoop.fs.s3a.secret.key", os.getenv("AWS_SECRET_ACCESS_KEY")) \
            .config("spark.hadoop.fs.s3a.path.style.access", "true") \
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
            .config("spark.hadoop.fs.s3a.threads.keepalivetime", "60") \
            .config("spark.hadoop.fs.s3a.connection.establish.timeout", "5000") \
            .config("spark.hadoop.fs.s3a.connection.timeout", "10000") \
            .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
            .config("spark.hadoop.fs.s3a.multipart.purge.age", "86400") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.postgresql:postgresql:42.6.0") \
            .getOrCreate()

try:
    spark = create_spark_session()
    print("✅ Spark Session criada com sucesso!")
    print(f"Versão: {spark.version}")
    
    # Teste rápido de conectividade Java (sem S3 ainda)
    print("Teste simples de processamento: ", spark.range(5).count())
    
except Exception as e:
    print("\n❌ ERRO FATAL CONTINUA:")
    print(e)
    # Dica extra de debug se falhar
    print("\n--- Dica de Debug ---")
    print("Se o erro persistir, verifique se seu container tem internet para baixar os JARs:")
    print("Execute no terminal: docker exec -it airflow curl -I https://repo1.maven.org")

⏳ Iniciando sessão Spark (Isso pode demorar se for o primeiro download dos JARs)...

❌ ERRO FATAL CONTINUA:
[JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

--- Dica de Debug ---
Se o erro persistir, verifique se seu container tem internet para baixar os JARs:
Execute no terminal: docker exec -it airflow curl -I https://repo1.maven.org
